In [7]:
import pickle
import pandas as pd
import numpy as np

In [2]:
data_df = pd.read_pickle("1-63-60-beam_DF.pkl")
MIP_df = pd.read_pickle("MIP_DF.pkl")
Ratio_df = pd.read_pickle("63_60_Ratio_DF.pkl")

In [11]:
mask = MIP_df["MIPs"]>0

print(max(np.array(MIP_df["MIPs"][mask])))
print(min(np.array(MIP_df["MIPs"][mask])))
print(np.mean(np.array(MIP_df["MIPs"][mask])))

4117.339667458432
2292.1615201900236
2965.803922211587


In [3]:
# Drop any conflicting columns before mergin
conflicting_mip_cols = set(data_df.columns).intersection(MIP_df.columns) - {"channel"}
conflicting_ratio_cols = set(data_df.columns).intersection(Ratio_df.columns) - {"channel"}

MIP_df = MIP_df.drop(columns=conflicting_mip_cols)
Ratio_df = Ratio_df.drop(columns=conflicting_ratio_cols)

# Merge with data_df
merged_df = data_df.merge(MIP_df, on="channel", how="left")
merged_df = merged_df.merge(Ratio_df, on="channel", how="left")

# Compute energy_MIP
merged_df["energy_MIP"] = merged_df["LG_ped_corr"] * merged_df["GainRatio"] / merged_df["MIPs"]

# Drop intermediate columns
merged_df.drop(columns=["MIPs", "GainRatio"], inplace=True)

print(merged_df.head())

   event  channel   HG   LG  CAEN_brd  CAEN_ch  CAEN  HG_ped_corr  \
0      0        0  119  102         0        0     0          0.0   
1      0        1  115  108         0        1     0          0.0   
2      0        2  108   94         0        2     0          0.0   
3      0        3  140   98         0        3     0          0.0   
4      0        4  142  108         0        4     0          0.0   

   LG_ped_corr      x     y    z  layer  layer_ch  energy_MIP  
0          0.0  -85.4 -85.4  0.0    0.0       0.0         0.0  
1          0.0  -36.6 -85.4  0.0    0.0       1.0         0.0  
2          0.0   12.2 -85.4  0.0    0.0       2.0         0.0  
3          0.0   61.0 -85.4  0.0    0.0       3.0         0.0  
4          0.0  109.8 -85.4  0.0    0.0       4.0         0.0  


In [4]:
with open('1-63-60-beam_DF.pkl', 'wb') as f:
    pickle.dump(merged_df, f)